In [ ]:
import os
import arcpy
import csv

In [ ]:
def count_valid_rows_and_extract_tags_source(shapefile_path):
    valid_row_count = 0
    tags_values = set()
    source_values = set()

    with arcpy.da.SearchCursor(shapefile_path, ['Latitude', 'Longitude', 'Tags', 'Source']) as cursor:
        for row in cursor:
            latitude, longitude, tags, source = row
            # Memeriksa apakah kolom Latitude dan Longitude tidak kosong dan tidak bernilai 0
            if latitude is not None or longitude is not None and (latitude != 0 or longitude != 0):
                valid_row_count += 1
                if tags:
                    tags_values.add(tags)
                if source:
                    source_values.add(source)

    return valid_row_count, ';'.join(tags_values), ';'.join(source_values)

def main(folder_path, output_folder):
    arcpy.env.workspace = folder_path
    
    output_csv = os.path.join(output_folder, "report_row.csv")
    
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Shapefile', 'Valid Rows', 'Tags', 'Source']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')  # Specify the delimiter as ';'
        writer.writeheader()

        for filename in arcpy.ListFiles("*.shp"):
            file_path = os.path.join(folder_path, filename)
            try:
                # Menghitung jumlah baris pada data shapefile yang memenuhi ketentuan dan ekstrak tags dan source
                valid_row_count, tags, source = count_valid_rows_and_extract_tags_source(file_path)

                # Skip rows where Latitude and Longitude are empty or equal to 0
                if valid_row_count > 0:
                    writer.writerow({'Shapefile': filename, 'Valid Rows': valid_row_count, 'Tags': tags, 'Source': source})
                    print(f"File {filename}: {valid_row_count} valid rows, Tags: {tags}, Source: {source}")
                else:
                    print(f"File {filename}: No valid rows found.")
            except Exception as e:
                print(f"Error while processing {filename}: {str(e)}")

if __name__ == "__main__":
    input_folder = r"D:\Magang ESRI\JOB\Cleansing data\Otomotif\add_tags_source"
    output_folder = r"D:\Magang ESRI\JOB\Cleansing data\Otomotif\check_row"
    main(input_folder, output_folder)